In [ ]:
# Python 3
import urllib.request
import re
import json
import base64
import pickle
import warnings
import urllib.parse
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup 
from bing_web_search_api import Client
batch_type = "xyzbatch" # a batch could be a code search word or a multi-level website / directory that your 
# search engine wants to index, i.e. a directory of gaming sub-websites
apikey = 'your Bing API key here'
exceptions = []

"""
# This is an unofficial Microsoft Bing api V.5 search/indexing engine bot demo code. 
# This sample code demonstrates how to (a) get results from Bing API (for which you need a valid license) and 
# (b) navigate to the website urls to extract indexing information for your custom search engine
# Sample use: build a custom search engine that indexes websites / blogs dedicated to sports cars collectors.
# Your search engine must comply with websites' TOS and robots policy.
"""

crms = {"xyzbatch":{
        "crm_base": "http://xyzbatch",
        "crm_email_base": "xyzbatch"
    }}

class SiteIndexer(object):
    def __init__(self):
        # source: https://gist.github.com/dideler/5219706
        self.regex = re.compile(("([a-z0-9!#$%&'*+\/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+\/=?^_`"
                    "{|}~-]+)*(@|\sat\s)(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?(\.|"
                    "\sdot\s))+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?)"))
        self.head = 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) \
                                                              Chrome/44.0.2403.155 Safari/537.36'

    def getEmails(self, textStr):
        return (email[0] for email in re.findall(self.regex, textStr) if not email[0].startswith('//'))

    def getImgString(self, url):
        req = urllib.request.Request(url, data=None, headers={'User-Agent': self.head})
        htmlfile = urllib.request.urlopen(req)
        htmltext = htmlfile.read()
        return base64.b64encode(htmltext)

    def readHTML(self,url):
        req = urllib.request.Request(url, data=None, headers={'User-Agent': self.head})
        htmlfile = urllib.request.urlopen(req)
        return htmlfile.read()
    
    def getSearchResults(self, query):
        # https://github.com/h-kanazawa/bing-web-search-api/blob/master/bing_web_search_api/client.py
        # https://msdn.microsoft.com/en-us/library/dn760794.aspx
        client = Client(apikey=apikey)
        response, content = client.search(count=50,q=query)
        search_results = json.loads(content.decode('utf-8')).get("webPages")
        sites = []
        for i in range(len(search_results["value"])):
            url = urllib.parse.urlparse(search_results.get("value")[i].get("displayUrl"), 'https').geturl()
            sites.append(url)
        return sites
    
    def getIndexedWebsites(self,sites):
        urls = {}
        for url in sites:
            try:
                if not "xyzbatch".lower() in url.lower():
                    exceptions.append(url)
                    continue
                crm_base = crms[batch_type].get("crm_base")
                crm_email_base = crms[batch_type].get("crm_email_base")
                if "http" not in url: url= crm_base + url
                htmltext = extractor.readHTML(url)
                soup = BeautifulSoup(htmltext)
                logo = extractor.getImgString(soup.find("img", {"id": "xyzbatch"}).get('src'))
                urls[url] = {}
                urls[url].update({"logo":logo})
                aurl  = soup.find_all("a", limit=250)
                for i, a in enumerate(aurl):
                    urls[url][i] = {}
                    href = a.get('href')
                    url_text = a.string.replace("\r\n","").strip() if not a.string is None else ""
                    if href is not None and len(href) > 0 and "mailto:" in href: 
                        urls[url][i].update({"email": href.replace("mailto:","")})  
                    elif href is not None and len(href) > 0 and "tel:" in href:
                        urls[url][i].update({"phone": href.replace("tel:","")})  
                    elif href is not None and len(href) > 0:
                        if "http" not in href: href= crm_base + href
                        urls[url][i].update({"url": href, "urltext":url_text})  
                        if "contacts" in url_text.lower():
                            htmltext = extractor.readHTML(href)
                            emails = [a for a in list(extractor.getEmails(str(htmltext))) if crm_email_base not in a]
                            urls[url][i].update({"contact_email": emails}) 
            except Exception as e:
                print(e)
                exceptions.append(url)
        return urls
        

if __name__ == "__main__":
    query = "your search query" #  Use combinatoric function itertools to generate a list of all possible search combinations
    # of target attributes, i.e. a collection of tuples in the shape of (attribute 1, attribute 2, attribute X)
    extractor = SiteIndexer()
    sites = extractor.getSearchResults(query) # Bing API v.5 # you can also limit results to the latest search data per Bing api specs
    #pickle.dump(sites, open( "sites.pkl", "wb" ) )
    #sites = pickle.load( open( "sites.pkl", "rb" ) )
    urls = extractor.getIndexedWebsites(sites) # Index Bing results by visiting the individual urls
    #print("exceptions",exceptions)
    print("done")    